In [11]:
from dotenv import load_dotenv
load_dotenv()

False

In [12]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [13]:
import lightning as L

from lightning.pytorch.callbacks import ModelCheckpoint

from src.model.modeling_instruction_tuning import LitInstructionModel
from src.data.dataset import get_train_dataloader, get_dev_dataloader, get_test_dataloader

In [14]:
SEED=42
DATASET_NAME = 'jwengr/M-ABSA-ko-noise'
MINI_BATCH_SIZE=4
N_BATCH = 4
BASE_MODEL_NAME='Qwen/Qwen3-0.6B'
EPOCHS=10
LEARNING_RATE = 5e-5
TRAIN_MAX_LENGTH=128
VALID_MAX_LENGTH=256
LORA_R=16
LORA_ALPHA=8


In [15]:
L.seed_everything(SEED)

Seed set to 42


42

In [16]:
train_dl = get_train_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=TRAIN_MAX_LENGTH)
dev_dl = get_dev_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=VALID_MAX_LENGTH)
test_dl = get_test_dataloader(DATASET_NAME, batch_size=1)

In [17]:
lit_model = LitInstructionModel(
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    lora_r=LORA_R,
    lora_alpha=LORA_ALPHA,
    epochs=EPOCHS
)

In [18]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/instruction_tuning',
    filename=f"{DATASET_NAME.split('/')[1]}/{BASE_MODEL_NAME.split('/')[1]}"+"-{epoch:02d}-{valid_loss:.4f}",
    every_n_epochs=1,
    save_top_k=-1,
)

In [19]:
trainer = L.Trainer(
    callbacks=[checkpoint_callback],
    precision='16-mixed',
    max_epochs=EPOCHS,
    enable_checkpointing=True,
    accumulate_grad_batches=N_BATCH
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(lit_model, train_dl, dev_dl)

/home/jjw1214/.local/share/virtualenvs/KROP-sezSv-b5/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:658: Checkpoint directory /home/jjw1214/github/KROP/checkpoints/instruction_tuning exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [4]

  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | PeftModelForCausalLM | 385 M  | train
-------------------------------------------------------
10.1 M    Trainable params
375 M     Non-trainable params
385 M     Total params
1,543.766 Total estimated model params size (MB)
1962      Modules in train mode
427       Modules in eval mode


Epoch 9: 100%|██████████| 3539/3539 [19:00<00:00,  3.10it/s, v_num=10]     

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 3539/3539 [19:02<00:00,  3.10it/s, v_num=10]
